In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from agno.agent import Agent
import datetime
from agno.models.openai import OpenAILike
import json
import httpx

#------------------ settings ------------------
api_key=os.getenv("QWEN_API_KEY")
base_url=os.getenv("QWEN_API_BASE_URL")
local_base_url = 'http://192.168.0.166:8000/v1'
local_model_name = 'Qwen3-235B'
model_name = 'qwen-plus-latest'

local_settings = {
  'api_key' : '123',
  'base_url' : local_base_url,
  'id' : local_model_name
}

qwen_settings = {
  'api_key' : api_key,
  'base_url' : base_url,
  'id' : model_name
}

settings = qwen_settings
#------------------ settings ------------------

ding_app_key = os.getenv('DING_APP_KEY')
ding_app_secret = os.getenv('DING_APP_SECRET')

In [48]:
def get_last_week_timestamps():
    # 获取当前时间
    now = datetime.datetime.now()
    
    # 计算今天是本周的第几天（周一为0，周日为6）
    today_weekday = now.weekday()
    
    # 计算上周日的日期（今天的日期 - 今天星期几 - 1天）
    last_sunday = now - datetime.timedelta(days=today_weekday + 1)
    
    # 计算上周一的日期（上周日 - 6天）
    last_monday = last_sunday - datetime.timedelta(days=6)
    
    # 设置上周一的时间为00:00:00
    last_monday_start = last_monday.replace(hour=0, minute=0, second=0, microsecond=0)
    
    # 设置上周日的时间为23:59:59.999999
    last_sunday_end = last_sunday.replace(hour=23, minute=59, second=59, microsecond=999999)
    
    # 转换为时间戳（秒级），然后乘以1000得到13位毫秒级时间戳
    monday_timestamp = int(last_monday_start.timestamp() * 1000)
    sunday_timestamp = int(last_sunday_end.timestamp() * 1000)
    
    # 格式化日期为YYYY-MM-DD
    monday_date_str = last_monday.strftime("%Y-%m-%d")
    sunday_date_str = last_sunday.strftime("%Y-%m-%d")
    
    return monday_timestamp, sunday_timestamp,monday_date_str,sunday_date_str

def get_access_token(appKey, appSecret):
    url = "https://api.dingtalk.com/v1.0/oauth2/accessToken"
    # JSON 请求体
    json_data = {
        "appKey":appKey,
        "appSecret":appSecret
    }
    # 发送 POST 请求
    response = httpx.post(
        url,
        json=json_data,  # 自动设置 Content-Type: application/json
        timeout=10.0  # 可选：设置超时
    )
    # 处理响应
    if response.status_code == 200:
        result = response.json()
        # print("请求成功:", result)  # 解析 JSON 响应
        return result['accessToken']
    else:
        print(f"请求失败，状态码: {response.status_code}, 错误: {response.text}")
        return None

def get_report(access_token, start_time, end_time):
    # 请求 URL
    url = "https://oapi.dingtalk.com/topapi/report/list?access_token=db37eafb549a3158a52711ed2dc80dd8"
    # JSON 请求体
    json_data = {
        "cursor": "0",
        "start_time": start_time,
        "template_name": "管理数字化工作周报",
        "size": 20,
        "end_time": end_time
    }

    # 发送 POST 请求
    response = httpx.post(
        url,
        json=json_data,  # 自动设置 Content-Type: application/json
        timeout=10.0  # 可选：设置超时
    )

    # 处理响应
    if response.status_code == 200:
        result = response.json()
        # print("请求成功:", result)  # 解析 JSON 响应
        return json.dumps(result, ensure_ascii=False)
    else:
        print(f"请求失败，状态码: {response.status_code}, 错误: {response.text}")
        return None

In [49]:
# 使用示例
monday_ts, sunday_ts,monday_date_str,sunday_date_str = get_last_week_timestamps()
access_token = get_access_token(appKey=ding_app_key, appSecret=ding_app_secret)
weekly_report = get_report(access_token, monday_ts, sunday_ts)

In [ ]:
weekly_report

In [50]:
agent = Agent(model=OpenAILike(**settings),description='你是一位周报总结专家.',instructions=['首先按照用户进行分组','根据用户的项目进行分组汇总项目事项','根据项目信息最后进行简短\客观\事实的分析总结','不要遗漏任何项目和项目事项信息',"严谨虚构和假设任何数据,没有请回答无记录",f'报告标题为:{monday_date_str}-{sunday_date_str}周报'],add_datetime_to_instructions=True,markdown=True,telemetry=False)

In [51]:
response = agent.run(message=weekly_report)

In [52]:
print(response.content)

# 2025-06-16 至 2025-06-22 周报

## 用户数据汇总

当前周报周期内，根据提供的记录：
- 没有用户提交相关的项目信息。
- 没有具体的任务或进展记录。

## 分析总结

本周报周期内无任何项目事项记录，无法进行进一步的分类汇总与分析。建议确认相关系统或数据源是否正常运行，或者是否有遗漏的信息需要补充。

如需进一步协助，请提供具体数据内容。
